This notebook takes the raw data found in multiple files within the "raw" folder and places them all into one file for a given month. The data is separated into motor vehicle theft and theft and finally geocoded so that it can be used in the map construction.

In [1]:
import pandas as pd
import os
import geocoder
import requests

In [2]:
for folder in os.listdir('./raw'):
    file_list = []
    for file in os.listdir('./raw/' + folder):
        if(file.endswith('.xls')):
            data = pd.read_excel('./raw/' + folder + '/' + file)
            file_list.append(data)
   
    df = pd.concat(file_list, ignore_index=True)
    
    columns = ['Incident Number','Date', 'Time', 'Police District','Offense 1',
          'Offense 2', 'Offense 3', 'Offense 4', 'Offense 5', 'Location']
    df.columns = columns
    
    df = df[df['Offense 1'].notnull()]
    df = df[df['Location'].notnull()]
    
    df.to_csv('./raw/' + folder + 'Complete.csv', sep=';', index = False)

In [3]:
def address_to_latlong(address):
    '''Returns the latitude and longitude for a given address'''
    address += ", Milwaukee, WI"
    location = geocoder.arcgis(address, session = session)
    
    if not location.latlng:
        print("Couldn't Parse: "  + address )
    else:
        print(f'Parsed: {address} - {location.latlng}')
    return location.latlng

In [4]:
for file in os.listdir('./raw/'):
    if(file.endswith('.csv')):
        df = pd.read_csv('./raw/'+ file, sep =';')
    
        zipped = zip(df['Offense 1'].items(), df['Offense 2'].items(), 
        df['Offense 3'].items(), df['Offense 4'].items(), df['Offense 5'].items())
        keyword = 'theft'
        
        for off_1, off_2, off_3, off_4, off_5 in zipped:
            if(not keyword in off_1[1].lower() and
              (pd.isnull(off_2[1]) or not keyword in off_2[1].lower()) and
              (pd.isnull(off_3[1]) or not keyword in off_3[1].lower()) and
              (pd.isnull(off_4[1]) or not keyword in off_4[1].lower()) and
              (pd.isnull(off_5[1]) or not keyword in off_5[1].lower())):
                df = df.drop(off_1[0])
 
        df['Address'] = df['Location'].copy()
            
        theft = df.copy()
        m_theft = df.copy()
        
        zipped = zip(df['Offense 1'].items(), df['Offense 2'].items(), 
        df['Offense 3'].items(), df['Offense 4'].items(), df['Offense 5'].items())
        keyword = 'motor vehicle theft'
        
        for off_1, off_2, off_3, off_4, off_5 in zipped:
            if(not keyword in off_1[1].lower() and
                (pd.isnull(off_2[1]) or not keyword in off_2[1].lower()) and
                (pd.isnull(off_3[1]) or not keyword in off_3[1].lower()) and
                (pd.isnull(off_4[1]) or not keyword in off_4[1].lower()) and
                (pd.isnull(off_5[1]) or not keyword in off_5[1].lower())):
                m_theft = m_theft.drop(off_1[0])
            else:
                theft = theft.drop(off_1[0])
                
        
        with requests.Session() as session:
            # This is likely to be very slow
            # depending on the geocoding service used
            theft['Location'] = theft['Location'].apply(address_to_latlong)
            m_theft['Location'] = m_theft['Location'].apply(address_to_latlong)
            
        theft.to_csv('./' + 'theft_' + file.split('Complete.csv')[0] + '.csv', sep=';', index = False)
        m_theft.to_csv('./' + 'm_theft_' + file.split('Complete.csv')[0] + '.csv', sep=';', index = False)
        print(f'Finished {file}')

Parsed: 6109 N 39TH ST, Milwaukee, WI - [43.129179586733216, -87.96094116475253]
Parsed: 4729 N 29TH ST, Milwaukee, WI - [43.103076577475576, -87.94915084744304]
Parsed: 4024 N 24TH PL, Milwaukee, WI - [43.0904342200935, -87.94344542627985]
Parsed: 4614 N TEUTONIA AV, Milwaukee, WI - [43.100993754659754, -87.94476059515867]
Parsed: 4242 N 26TH ST, Milwaukee, WI - [43.093753994171635, -87.945572930175]
Parsed: 7005 N TEUTONIA AV, Milwaukee, WI - [43.14513939494493, -87.95584821276456]
Parsed: 4847 N 22ND ST, Milwaukee, WI - [43.10512237121063, -87.9393286289432]
Parsed: 5103 N 31ST ST, Milwaukee, WI - [43.10991117346646, -87.95161063200395]
Parsed: 5256 N TEUTONIA AV, Milwaukee, WI - [43.1129616038293, -87.94999987753627]
Parsed: 5062 N 39TH ST, Milwaukee, WI - [43.10961283236193, -87.96140536078269]
Parsed: 3002 W SILVER SPRING DR, Milwaukee, WI - [43.11937143883067, -87.95184354948391]
Parsed: 4371 N 29TH ST, Milwaukee, WI - [43.096742947544584, -87.94931007372014]
Parsed: 1937 W HOPE

Parsed: 5600 W BURLEIGH ST, Milwaukee, WI - [43.0754108749512, -87.98311907123299]
Parsed: 3618 N 63RD ST, Milwaukee, WI - [43.08397171717931, -87.99085645181508]
Parsed: 2625 N 53RD ST, Milwaukee, WI - [43.06673716763805, -87.9798905809335]
Parsed: 6000 W MAIN ST, Milwaukee, WI - [43.025973504327844, -87.98765399999999]
Parsed: 1030 N 46TH ST, Milwaukee, WI - [43.04289541326679, -87.97098387131426]
Parsed: 7301 W NASH ST, Milwaukee, WI - [43.08440849913744, -88.00438193363932]
Parsed: 5120 W CENTER ST, Milwaukee, WI - [43.06800951875454, -87.97819147446477]
Parsed: 520 N 40TH ST, Milwaukee, WI - [43.03669045268045, -87.96425731845481]
Parsed: 588 S OAK PARK CT, Milwaukee, WI - [43.02646200000001, -87.98316162479058]
Parsed: 500 S 84TH ST, Milwaukee, WI - [43.02569908575227, -88.01717590230344]
Parsed: 6732 W FAIRVIEW AV, Milwaukee, WI - [43.03139646047075, -87.997122]
Parsed: 2860 N 74TH ST, Milwaukee, WI - [43.071109931444404, -88.00489937979704]
Parsed: 9250 W WISCONSIN AV, Milwauke

Parsed: 2057 S 14TH ST, Milwaukee, WI - [43.007285614338244, -87.92973109276667]
Parsed: 1670 S 11TH ST, Milwaukee, WI - [43.01274574136568, -87.92544788992879]
Parsed: 1670 S 11TH ST, Milwaukee, WI - [43.01274574136568, -87.92544788992879]
Parsed: 700 S 4TH ST, Milwaukee, WI - [43.02413665175822, -87.9153536517582]
Parsed: 100 W OREGON ST, Milwaukee, WI - [43.028412168431544, -87.91113316843153]
Parsed: 2481 S 6TH ST, Milwaukee, WI - [42.99972367055227, -87.91888955539827]
Parsed: 800 S 2ND ST, Milwaukee, WI - [43.02303266472387, -87.912606407958]
Parsed: 1209 S 18TH ST, Milwaukee, WI - [43.01878516763807, -87.93556856650677]
Parsed: 915 S 20TH ST, Milwaukee, WI - [43.021620167638076, -87.93809808482864]
Parsed: 2046 S 15TH PL, Milwaukee, WI - [43.00730013044608, -87.93223397735031]
Parsed: 1460 S MUSKEGO AV, Milwaukee, WI - [43.01600057588453, -87.93409120979305]
Parsed: 900 W HISTORIC MITCHELL ST, Milwaukee, WI - [43.012374148088995, -87.92278364808901]
Parsed: 1000 W GREENFIELD AV,

Parsed: 2309 E EUCLID AV, Milwaukee, WI - [42.987500876857695, -87.88127105743652]
Parsed: 2383 S WILLIAMS ST, Milwaukee, WI - [43.00084925541038, -87.8995131186497]
Parsed: 2352 E OKLAHOMA AV, Milwaukee, WI - [42.988176511541184, -87.87988333236193]
Parsed: 2506 S 6TH ST, Milwaukee, WI - [42.99897558090484, -87.91882538631792]
Parsed: 3348 S 12TH ST, Milwaukee, WI - [42.98355824562873, -87.9275739884588]
Parsed: 336 E DEWEY PL, Milwaukee, WI - [42.99175048600597, -87.905673]
Parsed: 1500 E SAVELAND AV, Milwaukee, WI - [42.9764474787926, -87.8907338068267]
Parsed: 332 E SCHILLER ST, Milwaukee, WI - [42.99317752596791, -87.9056608097409]
Parsed: 1400 E WARNIMONT AV, Milwaukee, WI - [42.97925753043997, -87.89191255536961]
Parsed: 2913 S 13TH ST, Milwaukee, WI - [42.991588524603685, -87.92879866803683]
Parsed: 2229 S 5TH ST, Milwaukee, WI - [43.00393617346646, -87.91694856261164]
Parsed: 4029 S AUSTIN ST, Milwaukee, WI - [42.97127383236193, -87.90714754097155]
Parsed: 3729 S WHITNALL AV, 

Parsed: 2460 N 27TH ST, Milwaukee, WI - [43.06377465889548, -87.94740241906649]
Parsed: 1240 W WALNUT ST, Milwaukee, WI - [43.052774626820096, -87.9288547642094]
Parsed: 2441 N 33RD ST, Milwaukee, WI - [43.063344754371286, -87.95526861368208]
Parsed: 1834 W WALNUT ST, Milwaukee, WI - [43.052712504327836, -87.93619272300768]
Parsed: 2800 W FOND DU LAC AV, Milwaukee, WI - [43.06847941445375, -87.94863309478991]
Parsed: 1350 W NORTH AV, Milwaukee, WI - [43.0604395199084, -87.92925372300768]
Parsed: 3345 W HIGHLAND BL, Milwaukee, WI - [43.04445253173897, -87.9565635]
Parsed: 2210 N 16TH ST, Milwaukee, WI - [43.059123387731546, -87.93260538242276]
Parsed: 1834 W WALNUT ST, Milwaukee, WI - [43.052712504327836, -87.93619272300768]
Parsed: 2410 N 30TH ST, Milwaukee, WI - [43.06273183236195, -87.95109291574828]
Parsed: 4202 W LLOYD ST, Milwaukee, WI - [43.057464525967916, -87.96604461226845]
Parsed: 3129 W LISBON AV, Milwaukee, WI - [43.054106694566535, -87.95345586079293]
Parsed: 2700 W BROWN 

...

Parsed: 2652 S 15TH PL #A, Milwaukee, WI - [42.996501136274446, -87.93250441185313]
Parsed: 4569 S 23RD ST, Milwaukee, WI - [42.96035123677765, -87.94382974092582]
Parsed: 1719 W LINCOLN AV, Milwaukee, WI - [43.00292551399403, -87.93517142200935]
Parsed: 4527 S 23RD ST, Milwaukee, WI - [42.96064465341363, -87.94382790303962]
Parsed: 2210 W COLLEGE AV, Milwaukee, WI - [42.93048427009659, -87.94276981918004]
Parsed: 545 W LAYTON AV, Milwaukee, WI - [42.95910947561956, -87.918471]
Parsed: 1822 W LINCOLN AV, Milwaukee, WI - [43.00301452596791, -87.93626341326677]
Parsed: 2300 W LAYTON AV, Milwaukee, WI - [42.959439511541206, -87.94405333236193]
Parsed: 4630 S 23RD ST, Milwaukee, WI - [42.96020623175025, -87.94373647677338]
Parsed: 6600 S 17TH ST, Milwaukee, WI - [42.92540186955392, -87.93669602842076]
Parsed: 3818 S 14TH ST, Milwaukee, WI - [42.97514358090484, -87.93025343738836]
Parsed: 2425 W CLAYTON CREST AV, Milwaukee, WI - [42.94767847403209, -87.94602016763807]
Parsed: 3844 S 20TH ST

Parsed: 5825 W HOPE AV, Milwaukee, WI - [43.09346551399403, -87.98505325145709]
Parsed: 5839 N 70TH ST, Milwaukee, WI - [43.12402216763806, -87.99848165753917]
Parsed: 5555 N 91ST ST, Milwaukee, WI - [43.118765444630384, -88.02601763258086]
Parsed: 5154 N 77TH ST, Milwaukee, WI - [43.11158981003384, -88.00744349674187]
Parsed: 8209 W BENDER AV, Milwaukee, WI - [43.13226548845881, -88.01343]
Parsed: 5957 N 70TH ST, Milwaukee, WI - [43.12618149708581, -87.99847010646873]
Parsed: 5730 N 91ST ST, Milwaukee, WI - [43.12211299417163, -88.02568487520942]
Parsed: 5290 N 82ND CT, Milwaukee, WI - [43.11399879519861, -88.01342003690353]
Parsed: 9025 W MILL RD, Milwaukee, WI - [43.13398253952925, -88.0249005]
Parsed: 5772 N 95TH ST, Milwaukee, WI - [43.12290343297363, -88.0307064190665]
Parsed: 9100 W APPLETON AV, Milwaukee, WI - [43.11284020845433, -88.02603183209959]
Parsed: 5230 N 91ST ST, Milwaukee, WI - [43.11326502943035, -88.0258138418839]
Parsed: 5663 N 65TH ST, Milwaukee, WI - [43.1208991

Parsed: 2700 W WISCONSIN AV, Milwaukee, WI - [43.038825181648335, -87.94779468164833]
Parsed: 626 N BROADWAY, Milwaukee, WI - [43.038127016100155, -87.9076652870912]
Parsed: 225 E MICHIGAN ST, Milwaukee, WI - [43.037407521207406, -87.90804210061168]
Parsed: 2621 W MICHIGAN ST, Milwaukee, WI - [43.037479521207395, -87.94705066763807]
Parsed: 2922 W JUNEAU AV, Milwaukee, WI - [43.04590446047076, -87.95080349999999]
Parsed: 2800 W CLYBOURN ST, Milwaukee, WI - [43.036207337280594, -87.94931898654636]
Parsed: 2900 W HIGHLAND BL, Milwaukee, WI - [43.044640457152525, -87.95089341618096]
Parsed: 509 N 32ND ST, Milwaukee, WI - [43.03642486372556, -87.9540605809335]
Parsed: 1141 N 24TH PL, Milwaukee, WI - [43.045336005828375, -87.94389906261164]
Parsed: 2134 W MICHIGAN ST, Milwaukee, WI - [43.03752147879262, -87.940494]
Parsed: 222 N 30TH ST, Milwaukee, WI - [43.03313874271453, -87.95156641185314]
Parsed: 1233 N 30TH ST, Milwaukee, WI - [43.0465947485429, -87.95134108814686]
Parsed: 520 N 28TH S

Parsed: 3750 N HUMBOLDT BL, Milwaukee, WI - [43.08500705245544, -87.89745588242278]
Parsed: 2338 W KEEFE AV, Milwaukee, WI - [43.0824715226497, -87.94227116472388]
Parsed: 300 W CENTER ST, Milwaukee, WI - [43.06761716688744, -87.91411124769016]
Parsed: 418 E GARFIELD AV, Milwaukee, WI - [43.05907152264969, -87.90610988773155]
Parsed: 1407 N MARTIN L KING JR DR, Milwaukee, WI - [43.049288988314586, -87.91438864311245]
Parsed: 2463 N PALMER ST, Milwaukee, WI - [43.06341586372554, -87.90959912868573]
Parsed: 2541 N 5TH ST, Milwaukee, WI - [43.064677094397794, -87.91706816840232]
Parsed: 3857 N 18TH ST, Milwaukee, WI - [43.087095419095164, -87.93330015364403]
Parsed: 3533 N 25TH ST, Milwaukee, WI - [43.083242696087495, -87.9448490809335]
Parsed: 3857 N 24TH ST, Milwaukee, WI - [43.08732928282072, -87.94205957761528]
Parsed: 3300 N MARTIN L KING JR DR, Milwaukee, WI - [43.07894310574758, -87.91611958171289]
Parsed: 401 E CAPITOL DR, Milwaukee, WI - [43.08916851889972, -87.9060000693923]
Par

Parsed: 1935 S 31ST ST, Milwaukee, WI - [43.009497754371296, -87.95284159925535]
Parsed: 2066 S 19TH ST, Milwaukee, WI - [43.00704189113227, -87.9366744065024]
Parsed: 2513 W GREENFIELD AV, Milwaukee, WI - [43.016958546742615, -87.94554266472387]
Parsed: 1234 S 29TH ST, Milwaukee, WI - [43.01820916763808, -87.95021288631791]
Parsed: 1702 S LAYTON BL, Milwaukee, WI - [43.01235896863639, -87.94778545902845]
Parsed: 923 S 31ST ST, Milwaukee, WI - [43.02165133943519, -87.95261953376293]
Parsed: 2220 W BURNHAM ST, Milwaukee, WI - [43.010345482110836, -87.94192008090484]
Parsed: 3135 W FOREST HOME AV, Milwaukee, WI - [42.99717472271922, -87.9540368166658]
Parsed: 1957 S 36TH ST, Milwaukee, WI - [43.00902117346644, -87.9589690010096]
Parsed: 772 S 26TH ST, Milwaukee, WI - [43.02266399999999, -87.94631337578633]
Parsed: 2110 W NATIONAL AV, Milwaukee, WI - [43.02269045325738, -87.93993362629114]
Parsed: 3423 W NATIONAL AV, Milwaukee, WI - [43.02162452842076, -87.95707851679265]
Parsed: 2100 W P